###Problem Statement
The table logs the spendings history of users that make purchases from an online shopping website which has a desktop and a mobile application.
(user_id, spend_date, platform) is the primary key of this table.
The platform column is an ENUM type of ('desktop', 'mobile').
Write an SQL query to find the total number of users and the total amount spent using mobile only, desktop only and both mobile and desktop together for each date.


###Pyspark

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum, countDistinct

# Fill in missing combinations with zero values
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from itertools import product

# Create a Spark session
spark = SparkSession.builder.appName("SpendingData").getOrCreate()

# Define the schema
schema = ["user_id", "spend_date", "platform", "amount"]

# Create the data
data = [
    (1, "2019-07-01", "mobile", 100),
    (1, "2019-07-01", "desktop", 100),
    (2, "2019-07-01", "mobile", 100),
    (2, "2019-07-02", "mobile", 100),
    (3, "2019-07-01", "desktop", 100),
    (3, "2019-07-02", "desktop", 100),
]

# Create a DataFrame
df = spark.createDataFrame(data, schema)

# Convert spend_date to datetime format
df = df.withColumn("spend_date", to_date(df.spend_date, "yyyy-MM-dd"))

# Identify users using both platforms on the same day
df_both = df.groupBy("user_id", "spend_date").agg(
    countDistinct("platform").alias("platform_count")
)
df_both = df_both.withColumn(
    "new_platform", when(col("platform_count") == 2, "both").otherwise("single")
)

# Join the original DataFrame with the df_both DataFrame to get the new_platform column
df = df.join(
    df_both.select("user_id", "spend_date", "new_platform"),
    on=["user_id", "spend_date"],
    how="left",
)

# Assign 'mobile' or 'desktop' to new_platform for single platform users
df = df.withColumn(
    "new_platform",
    when(col("new_platform") == "single", col("platform")).otherwise(
        col("new_platform")
    ),
)

# Aggregate the data
result_df = df.groupBy("spend_date", "new_platform").agg(
    sum("amount").alias("total_amount"), countDistinct("user_id").alias("no_of_users")
)

# Create a DataFrame with all possible combinations of spend_date and new_platform
dates = df.select("spend_date").distinct().rdd.flatMap(lambda x: x).collect()
platforms = ["both", "desktop", "mobile"]
all_combinations = spark.createDataFrame(
    [(date, platform) for date, platform in product(dates, platforms)],
    ["spend_date", "new_platform"],
)

# Join the result with all_combinations DataFrame to fill missing combinations
final_df = all_combinations.join(
    result_df, on=["spend_date", "new_platform"], how="left"
).fillna(0)

# Show the final result
final_df.orderBy("spend_date", "new_platform").display()

spend_date,new_platform,total_amount,no_of_users
2019-07-01,both,200,1
2019-07-01,desktop,100,1
2019-07-01,mobile,100,1
2019-07-02,both,0,0
2019-07-02,desktop,100,1
2019-07-02,mobile,100,1


###Saprk SQL

In [0]:
# Create temporary view
df.createOrReplaceTempView("spending")

spark.sql("SELECT * FROM spending").display()

# Display the content of the table

user_id,spend_date,platform,amount
1,2019-07-01,mobile,100
1,2019-07-01,desktop,100
2,2019-07-01,mobile,100
2,2019-07-02,mobile,100
3,2019-07-01,desktop,100
3,2019-07-02,desktop,100


In [0]:
%sql
with platform_track as (
  select
    user_id,
    spend_date,
    count(distinct platform) as no_of_platform_used
  from
    spending
  group by
    user_id,
    spend_date
),
platform_sub as (
  select
    sp.spend_date,
    sp.user_id,
    sp.platform,
    sp.amount,case
      when pl.no_of_platform_used > 1 then 'both'
      else sp.platform
    end as platform_used
  from
    spending sp
    left join platform_track pl on pl.user_id = sp.user_id
    and pl.spend_date = sp.spend_date
  union all
  select
    distinct spend_date,
    null as user_id,
    null as platform,
    0 as amount,
    'both' as platform_used
  from
    spending
)
select
  spend_date,
  platform_used,
  sum(amount) as total_amount,
  count(distinct user_id) as total_users
from
  platform_sub
group by
  platform_used,
  spend_date
order by
  spend_date,
  platform_used desc

spend_date,platform_used,total_amount,total_users
2019-07-01,mobile,100,1
2019-07-01,desktop,100,1
2019-07-01,both,200,1
2019-07-02,mobile,100,1
2019-07-02,desktop,100,1
2019-07-02,both,0,0
